Created on Mon Oct 11 14:54:29 2021

@author: Gregoire, Adrien

# Imports

In [1]:
import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [2]:
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from matplotlib.collections import PolyCollection
from matplotlib import colors as mcolors
from matplotlib.patches import FancyArrowPatch

def polygon_under_graph(xlist, ylist):
    '''
    Construct the vertex list which defines the polygon filling the space under
    the (xlist, ylist) line graph.  Assumes the xs are in ascending order.
    '''
    return [(xlist[0], 0.), *zip(xlist, ylist), (xlist[-1], 0.)]
def cc(arg,alpha):
    '''
    Shorthand to convert 'named' colors to rgba format at 60% opacity.
    '''
    return mcolors.to_rgba(arg, alpha)

In [3]:
%matplotlib
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})

Using matplotlib backend: TkAgg


In [4]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

hbar = 1.0545718e-34
c = 299792458

N = 1024

# Functions used to describe ions

$\texttt{MB}$ is the Maxwell-Boltzmann distribution. It provides the probability to find an atom with a given velocity in a gas with temperature T.

$\texttt{pfl_dop} = \rho_{ee}$ is the atomic ray profile with Doppler effect. It provides the probability of excitation given Rabi frequency, detuning, lambda and velocity of atom. It is considered equal to the excited population, i.e the proportion of excited atoms.

$ \texttt{MB} = \sqrt{\frac{m}{2\pi k_BT}}\exp{-\frac{mv^2}{2k_BT}}$

$\texttt{pfl_dop} = \rho_{ee} = \frac{0.25\Omega_R^2}{0.5\Omega_R^2 + 0.25\Gamma^2 + \Delta^2} = \frac{A/2}{A+B + \Delta^2}$

In [5]:
# Doppler profile
def pfl_dop(v, delta, k, Rab, Gam):
    return .25*Rab**2/(0.5*Rab**2+.25*Gam**2+(delta-k*v)**2)

# Maxwell-Boltzmann distribution
def MB(v, T):
    kb = 1.38e-23
    m_Ca = 40*1.66e-27
    return (m_Ca/(np.pi*2*kb*T))**(1/2) * np.exp(-m_Ca*v**2/(2*kb*T)) # **1 car vitesse par rapport au laser compte seulement (1D)

# Probabilities product
# For a range of velocities,
# compute the prob_fluo
# for a given T, delta, Rab, Gam
def prob_fluo(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)

def T_lim(delta):
    return -0.5*hbar*Gam**2*(1+(2*delta/Gam)**2) / (4*delta)/kb

In [6]:
# Laser parameters
k = 15826663 # m^-1
Gam = 21570000.0 *2*pi
delta = Gam
lam = 397e-9 # m
I = 170 # W/m²

# Fluorescence

For a single ion the fluorescence $F \propto \rho$. For an ensemble of ions, the fluorescence F is related to the product of both probabilities $ F \propto \texttt{MB} \times \texttt{pfl_dop} = \texttt{prob_fluo}$. Maxwell-Boltzmann is not necessarily the best to describe the cold crystal ensemble but lets use it. This product is still a probability, lets call it $P$. $dP/dT = 0$ gives us the temperature for which the fluo is maximum, $d^2P/dT^2 = 0$ gives us the temperature for which the fluorescence variation  $dP/dT$ is maximum.

### Analytically with $\rho$

$\frac{d\rho}{dv} = \frac{A}{2}\frac{2\sqrt{z}}{(C+z)^2}, z=(\delta-v)^2, C = A + B$

$\frac{d^2\rho}{dv^2} = \frac{f}{g} = [-2(C+z)^2 + 8z(C+z)]\frac{A}{2(\dots)^4}$

The idea is to search for the setting where the variation of fluorescence is the highest for a given input in energy. The energy introduced by the GiantMolecule is brought under the form of thermal energy. Thus we want to set the cloud in a configuration with the best variation of fluorescence for a variation in temperature. There are two ways of achieving this. Considering infinitesimal differences, this requires to compute $\frac{d^2\rho}{dv^2} = 0$.

$\frac{d^2\rho}{dv^2} = 0 \Rightarrow f = 0 \Rightarrow z = \frac{1}{3}C \Rightarrow \delta - v = \pm \sqrt{\frac{C}{3}} = \pm \sqrt{\frac{A+B}{3}} $

Considering the thermal velocity $v = \sqrt{2k_BT/m}$ it is possible to rewrite $v = \delta - z$  as $2k_BT/m = (\delta - z)^2$ then $T = \frac{m(\delta - z)^2}{(2k_B)}\frac{1}{k^2}$ 

The above computation is done just below

In [7]:
# T = 0.005
# v = sqrt(2*kb*T/m_Ca)
# s = 2 # dans le code 2 correspond a omega_rabi=Gam
# s0 = 1
omega_rabi = Gam # sqrt(2*s*(0.25*Gam**2+(delta-k*v)**2))
# omega_rabi = sqrt( 2*s0*0.25*Gam**2 )
A = 0.5*omega_rabi**2
B = 0.25*Gam**2
zz = sqrt( (A+B)/3 )
Tplus = m_Ca*(delta-zz)**2/(2*kb)/k**2
Tmoins = m_Ca*(delta+zz)**2/(2*kb)/k**2
print(Tplus*1e3,'mK')
print(Tmoins*1e3,'mK')

print('Seems OK to me when looking data from simu article')

44.1838525472236 mK
397.65467292501233 mK
Seems OK to me when looking data from simu article


# Numerical computation of fluorescence

### Below some plots in order to see the fluo of a single ion

In [8]:
# Plot the pfl-dop probability vs Temperature

Tmin = .0005
Tmax = 10
vmin = sqrt(2*kb*Tmin/m_Ca)
vmax = sqrt(2*kb*Tmax/m_Ca)
nu = -k*np.linspace(vmin,vmax,1000)
dv = mean(diff(nu))/k

Rab = Gam # or np.sqrt(3*lam**3*Gam/(4*np.pi**2*hbar*c) * I) 

pfl0 = pfl_dop(nu/k, 0, k, Rab, Gam)
pfl05 = pfl_dop(nu/k, -Gam/2, k, Rab, Gam)
pfl1 = pfl_dop(nu/k, -Gam, k, Rab, Gam)
pfl2 = pfl_dop(nu/k, +Gam, k, Rab, Gam)
# pfl3 = pfl_dop(nu/k, +3*Gam, k, Rab, Gam)

fig = plt.figure('Dop prof',clear='True')
ax = fig.add_subplot(111)
ax.plot((nu/k)**2*m_Ca/2/kb, pfl1, '-',color='xkcd:red',label=r'$\delta = -\Gamma$')
ax.plot((nu/k)**2*m_Ca/2/kb, pfl05, ':',color='xkcd:black',label=r'$\delta = -\frac{1}{2}\Gamma$')
ax.plot((nu/k)**2*m_Ca/2/kb, pfl0, '-',color='xkcd:green',label=r'$\delta = 0$')
ax.plot((nu/k)**2*m_Ca/2/kb, pfl2, '-',color='xkcd:blue',label=r'$\delta = \Gamma$')
# ax.plot((nu/k)**2*m_Ca/2/kb, pfl3, '-',color='xkcd:black',label=r'$\delta = \Gamma$')
# ax.vlines(0,0,1)

# ax.arrow(-Gam*1e-6+10, 1.05,
#          Gam*1e-6-20, 0,
#          color='xkcd:rouge',
#          width=1e-2,head_width=5e-2,head_length=10,zorder=100)
# ax.arrow(-10, 1.05,
#          -Gam*1e-6+20, 0,
#          color='xkcd:rouge',
#          width=1e-2,head_width=5e-2,head_length=10)

ax.set_xlabel(r'T [K]')
ax.set_ylabel(r'Probability $\rho_{ee}$')
ax.set_xlim(-0.05,1.35)
ax.set_ylim(-0.05,0.38)
ax.annotate(r'$\Gamma$', xy=(-Gam*1e-6/2,1.075), xycoords='data',
    size=25, ha='center', va='bottom', color='xkcd:rouge')

ax.grid()
ax.legend(title='detuning',loc=1,fontsize=15)

tight_layout()

H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 


In [9]:
# Plot the pfl-dop fluorescence vs Temperature

Tmin = .0005
Tmax = 10
vmin = sqrt(2*kb*Tmin/m_Ca)
vmax = sqrt(2*kb*Tmax/m_Ca)
nu = -k*np.linspace(vmin,vmax,1000)
dv = mean(diff(nu))/k

Rab = Gam # or np.sqrt(3*lam**3*Gam/(4*np.pi**2*hbar*c) * I) 

pfl0 = pfl_dop(nu/k, 0, k, Rab, Gam)*Gam     *1e-6
pfl05 = pfl_dop(nu/k, -Gam/2, k, Rab, Gam)*Gam     *1e-6

pfl1 = pfl_dop(nu/k, -Gam, k, Rab, Gam)*Gam  *1e-6
pfl2 = pfl_dop(nu/k, +Gam, k, Rab, Gam)*Gam  *1e-6
# pfl3 = pfl_dop(nu/k, +3*Gam, k, Rab, Gam)

fig = plt.figure('Dop prof fluo tot',clear='True')
ax = fig.add_subplot(111)
ax.plot((nu/k)**2*m_Ca/2/kb, pfl1, '-',color='xkcd:red',label=r'$\delta = -\Gamma$')
ax.plot((nu/k)**2*m_Ca/2/kb, pfl05, ':',color='xkcd:black',label=r'$\delta = -\frac{1}{2}\Gamma$')
ax.plot((nu/k)**2*m_Ca/2/kb, pfl0, '-',color='xkcd:green',label=r'$\delta = 0$')
ax.plot((nu/k)**2*m_Ca/2/kb, pfl2, '-',color='xkcd:blue',label=r'$\delta = \Gamma$')
# ax.plot((nu/k)**2*m_Ca/2/kb, pfl3, '-',color='xkcd:black',label=r'$\delta = \Gamma$')

ax.hlines(pfl1[0]-sqrt(pfl1[0]),0,1,color='xkcd:red',ls=':',lw = 3)
ax.hlines(pfl1[0]+sqrt(pfl1[0]),0,1,color='xkcd:red',ls=':',lw = 3)
ax.hlines(pfl0[0]-sqrt(pfl0[0]),0,0.1,color='xkcd:green',ls=':',lw = 3)
ax.hlines(pfl2[0]-sqrt(pfl2[0]),0,0.1,color='xkcd:blue',ls=':',lw = 3)

ax.set_xlabel(r'T [K]')
ax.set_ylabel(r'Fluorescence $F$ [cts/µs]')
ax.set_xlim(-0.05,1.55)
# ax.set_ylim(-0.05,0.55)
ax.grid()
ax.legend(title='detuning',loc=1,fontsize=15)

plt.tight_layout()
subplots_adjust(top=0.96,
bottom=0.135,
left=0.115,
right=0.965,
hspace=0.2,
wspace=0.2)

### Check the derivates

In [225]:
# Plot the pfl-dop fluorescence derivates vs Temperature

Tmin = .0005
Tmax = 10
vmin = sqrt(2*kb*Tmin/m_Ca)
vmax = sqrt(2*kb*Tmax/m_Ca)
nu = -k*np.linspace(vmin,vmax,1000)
dv = mean(diff(nu))/k

Rab = Gam # or np.sqrt(3*lam**3*Gam/(4*np.pi**2*hbar*c) * I) 

pfl1 = pfl_dop(nu/k, -Gam, k, Rab, Gam)*Gam  *1e-6
# pfl3 = pfl_dop(nu/k, +3*Gam, k, Rab, Gam)

fig = plt.figure('Dop prof fluo tot deriv',clear='True')
ax = fig.add_subplot(111)
ax.plot((nu/k)**2*m_Ca/2/kb, pfl1, '-',color='xkcd:red',label=r'$F$')
ax.plot((nu[:-1]/k)**2*m_Ca/2/kb, diff(pfl1), '--',color='xkcd:red',label=r'$dF/dT$')
ax.plot((nu[:-2]/k)**2*m_Ca/2/kb, diff(pfl1,n=2), ':',color='xkcd:red',label=r'$d^2F/dT^2$')
# ax.plot((nu/k)**2*m_Ca/2/kb, pfl3, '-',color='xkcd:black',label=r'$\delta = \Gamma$')

ax.set_xlabel(r'T [K]')
# ax.set_ylabel(r'$dF/dT$')
# ax.set_xlim(-0.05,1.55)
# ax.set_ylim(-0.05,0.55)


ax.grid()
ax.legend(title='detuning',loc=1,fontsize=15)

plt.tight_layout()
subplots_adjust(top=0.96,
bottom=0.135,
left=0.115,
right=0.965,
hspace=0.2,
wspace=0.2)

### Fluo for an ensemble of ions

In [8]:
# Plot the MBxpfl-dop vs Temperature

Tmin = .0005
Tmax = 10
vmin = sqrt(2*kb*Tmin/m_Ca)
vmax = sqrt(2*kb*Tmax/m_Ca)
nu = -k*np.linspace(vmin,vmax,1000)
dv = mean(diff(nu))/k

Rab = Gam # or np.sqrt(3*lam**3*Gam/(4*np.pi**2*hbar*c) * I) 

vmoins = -20
vplus = 20
nv = 1000
dv = (vplus-vmoins)/nv

pfl0 = []
pfl05 = []
pfl1 = []
pfl2 = []

Temps = linspace(0.005,10,1000)
for T in Temps:
    pfl0.append( sum(prob_fluo(vmoins, vplus, nv, T, 0, k, Rab, Gam)[1])*dv*Gam * N )
    pfl05.append( sum(prob_fluo(vmoins, vplus, nv, T, -Gam/2, k, Rab, Gam)[1])*dv*Gam * N )
    pfl1.append( sum(prob_fluo(vmoins, vplus, nv, T, -Gam, k, Rab, Gam)[1])*dv*Gam * N )
    pfl2.append( sum(prob_fluo(vmoins, vplus, nv, T, Gam, k, Rab, Gam)[1])*dv*Gam * N )
# pfl3 = pfl_dop(nu/k, +3*Gam, k, Rab, Gam)

fig = plt.figure('MBxDop prof fluo tot',clear='True')
ax = fig.add_subplot(111)

ax.plot(Temps, array(pfl0)*1e-9, '-',color='xkcd:green',label=r'$\delta = 0$')
ax.plot(Temps, array(pfl05)*1e-9, ':',color='xkcd:black',label=r'$\delta = -\Gamma/2$')
ax.plot(Temps, array(pfl2)*1e-9, '-',color='xkcd:blue',label=r'$\delta = \Gamma$')
ax.plot(Temps, array(pfl1)*1e-9, '--',color='xkcd:red',label=r'$\delta = -\Gamma$')
# ax.plot((nu/k)**2*m_Ca/2/kb, pfl3, '-',color='xkcd:black',label=r'$\delta = \Gamma$')

ax.hlines((pfl2[0]-sqrt(pfl2[0]))*1e-9,0,1,color='xkcd:red',ls=':',lw = 3)
ax.hlines((pfl2[0]+sqrt(pfl2[0]))*1e-9,0,1,color='xkcd:red',ls=':',lw = 3)
# ax.hlines(pfl0[0]-sqrt(pfl0[0]),0,0.1,color='xkcd:green',ls=':',lw = 3)
# ax.hlines(pfl05[0]-sqrt(pfl05[0]),0,0.1,color='xkcd:blue',ls=':',lw = 3)


ax.set_xlabel(r'T [K]')
ax.set_ylabel(r'Fluorescence $F$ [kcts/µs]')
ax.set_xlim(-0.02,2.22)
# ax.set_ylim(12,46)

ax.grid()
ax.legend(title='detuning',loc=1,fontsize=15)

plt.tight_layout()
subplots_adjust(top=0.96,
bottom=0.135,
left=0.115,
right=0.965,
hspace=0.2,
wspace=0.2)

In [9]:
# comparison formula single ion, and MB

Tmin = .0005
Tmax = 10
vmin = sqrt(2*kb*Tmin/m_Ca)
vmax = sqrt(2*kb*Tmax/m_Ca)
nu = -k*np.linspace(vmin,vmax,1000)
dv = mean(diff(nu))/k

Rab = Gam # or np.sqrt(3*lam**3*Gam/(4*np.pi**2*hbar*c) * I) 

vmoins = -50
vplus = 50
nv = 1000
dv = (vplus-vmoins)/nv

pfl0 = pfl_dop(nu/k, -Gam, k, Rab, Gam)*Gam


pfl1 = []
Temps = linspace(0.005,10,1000)
for T in Temps:
    pfl1.append( sum(prob_fluo(vmoins, vplus, nv, T, -Gam, k, Rab, Gam)[1])*dv*Gam * N )
# pfl3 = pfl_dop(nu/k, +3*Gam, k, Rab, Gam)

fig = plt.figure('MBxDop vs single',clear='True')
ax = fig.add_subplot(111)

# ax.plot(Temps, array(pfl0)*1e-9, '-',color='xkcd:green',label=r'single')
ax.plot((nu/k)**2*m_Ca/2/kb, pfl0/max(pfl0), '-',color='xkcd:red',label=r'Single ion (Eq. 4.12)')
ax.plot(Temps, array(pfl1)/max(pfl1), '--',color='xkcd:blue',label=r'N ions with MB (Eq. 4.24)')
# ax.plot((nu/k)**2*m_Ca/2/kb, pfl3, '-',color='xkcd:black',label=r'$\delta = \Gamma$')

ax.set_xlabel(r'T [K]')
ax.set_ylabel(r'Norm. fluo. $F$ [a.u]')
ax.set_xlim(-0.1,10.1)
# ax.set_ylim(12,46)

ax.grid()
ax.legend(title='',loc=1,fontsize=15)

plt.tight_layout()
# subplots_adjust(top=0.96,
# bottom=0.16,
# left=0.115,
# right=0.965,
# hspace=0.2,
# wspace=0.2)

In [10]:
# Plot the MB distribution and \rho

T = .001
nu = k*np.linspace(-20,20,10000)
dv = mean(diff(nu))/k
y = MB(nu/k, T)

Rab = Gam # or np.sqrt(3*lam**3*Gam/(4*np.pi**2*hbar*c) * I) 

pfl = pfl_dop(nu/k, delta, k, Rab, Gam)
pfl2 = pfl_dop(nu/k, -delta, k, Rab, Gam)

fig = plt.figure('MB and Dop prof',clear='True')
ax = fig.add_subplot(111)
ax.plot(nu*1e-6, y/np.max(y),label=f'{T*1e3:.1f} mK MB profile')
ax.plot(nu*1e-6, pfl/np.max(pfl), '-',label=r'$\rho_{ee}$ (counter-propagating)')
ax.plot(nu*1e-6, pfl2/np.max(pfl2), '-',label=r'$\rho_{ee}$ (co-propagating)')

ax.arrow(-Gam*1e-6+10, 1.05,
         Gam*1e-6-20, 0,
         color='xkcd:rouge',
         width=1e-2,head_width=5e-2,head_length=10,zorder=100)
ax.arrow(-10, 1.05,
         -Gam*1e-6+20, 0,
         color='xkcd:rouge',
         width=1e-2,head_width=5e-2,head_length=10)

ax.set_xlabel(r'$\nu = k v$ [MHz]')
ax.set_ylabel(r'Probability')
ax.set_ylim(-0.05,1.35)
ax.annotate(r'$\Gamma$', xy=(-Gam*1e-6/2,1.075), xycoords='data',
    size=25, ha='center', va='bottom', color='xkcd:rouge')

below = []
for i in range(len(y)):
    if y[i]/np.max(y)>pfl[i]/np.max(pfl):
        below.append(pfl[i]/np.max(pfl))
    else:
        below.append(y[i]/np.max(y))
verts1 = []
verts1.append(polygon_under_graph(nu*1e-6,below))

poly1 = PolyCollection(verts1, facecolors=cc('xkcd:purplish blue',0.85), hatch = '..')
ax.add_collection(poly1)

ax.grid()
ax.legend(loc=1,fontsize=15)

tight_layout()

In [215]:
# >>> Takes few minutes <<<

vmin = -20 # from 
vmax = 20  # to
nv = 1000  # resolution

temps = np.linspace(.00055, 5, 2000)  # temperatures tested
deltas = np.linspace(-1,1, 51)*Gam # detunings tested
z = [] # z is the integral MBxDopProfile : z[T,delta]
for i, temp in enumerate(temps): # for a set of T
    z.append([])    
    for j, delta in enumerate(deltas): # for a set of deltas
        # dop is the product MB*\rho in function of v
        # dop is an array with length nv
        # for each iteration it is computed for one T, one delta
        # and a range of velocities from vmin to vmax
        nunu, dop = prob_fluo(vmin, vmax, nv, temp, delta, k, Rab, Gam)
        
        dv = mean(diff(nunu))/k
        # then I integrate the product over v
        z[i].append(np.sum(dop)*dv) # the integrals are normalised
                                     # by dv the step in v 
z = array(z)

# What the above loop do
Just to be clear

$\texttt{prob_fluo}$ computes the product MB*\rho

with MB and \rho plotted against velocity ranging from vmin to vmax. At every loop step, it is computed for one given T and delta, Rab, Gam.

The result of one computation is an integral, stored in `z[T][delta]`

Then the loop changes delta then T and in the end all the integrals are stored in `z[T][delta]`

Below is an example of z[:][delta], a curve of fluo in function of T, for a given delta. It is possible to write $F \propto$ `z[T][delta]` or again $F(T) \propto$ `z[:][delta]`, or $F(\delta) \propto$ `z[T][:]`.

In [234]:
# The temperature
# For which fluo is max
temps = np.linspace(.00055, 5, 1000)
j = 0
print('Detuning',deltas[j]/Gam,'$\Gamma$')
print('Max fluo for T =',temps[argmax(z[:,j])]*1e3,'mK')
print('Max fluo var for T =',temps[argmin(diff(abs(z[:,j]),n=2))]*1e3,'mK')

Detuning -1.0 $\Gamma$
Max fluo for T = 170.70145145145148 mK
Max fluo var for T = 25.572272272272276 mK


In [224]:
# Plot z
# the excited population

fig = plt.figure('ex curve 0',clear='True')
ax = fig.add_subplot(111)
# plt.title('Optimal temperature')
for_detuning = 0
print(deltas[for_detuning]/Gam)

ax.plot(temps[:-2]*1e3, diff(z[:,for_detuning],n=2),
        marker='',ls='-',color = 'blue')

ax.set_xlabel('Temperature [mK]')
ax.set_ylabel('P')
ax.grid()
# ax.legend(fontsize=15)
tight_layout()

-1.0


# Numerical derivation of derivate

Then, $\frac{dF}{dT} = $ `diff(np.array(z)[:,i])` is the fluo variation in function of temperature, and $\frac{d^2F}{dT^2} = $ `diff(np.array(z)[:,i],n=2)` the second order derivate of fluo against temperature. 

Below are computed the temperature for which $\frac{dF}{dT} = 0$ (in fact when $|\frac{dF}{dT}|$ is minimum), for all the tested detunings. Also when $\frac{d^2F}{dT^2} = 0$ ($|\frac{d^2F}{dT^2}|$ minimum).

In [237]:
# Search integrals maximum
# with derivative method
# pos of optimal temp

# z[temperature,detuning] is the integral MBxDopProfile

n = np.shape(z)[1]
T0 = []
F0 = []
for i in np.arange(0,n,1):
    T0.append([])
    F0.append([])
    # Temperature
    # when first and second derivate are 0 (minimum abs)
    T0[i].append( temps[ argmin(abs(diff(np.array(z)[:,i]))) ] )
    T0[i].append( temps[ argmin(abs(diff(np.array(z)[:,i],n=2))) ] )
    # the fluo itself
    # when the two above conditions are fullfiled
    F0[i].append(array(z)[argmin(abs(diff(np.array(z)[:,i]))),i])
    F0[i].append(array(z)[argmin(abs(diff(np.array(z)[:,i],n=2))),i])
    
T0 = array(T0)
F0 = array(F0)

# True_min = T0[:,1]
# True_max = T0[:,2]
# True_deltas = hstack((deltas[:],deltas[:]))
# True_true = hstack((True_min,True_max))

# Frue_min = F0[:,1]
# Frue_max = F0[:,2]
# Frue_true = hstack((Frue_min,Frue_max))

# ou Fluo est max c'est le min de la valeur absolue de diff(z)

In [238]:
# Plot Temperature
# for maximum fluo
# for max fluo var
# For all detunings

fig = plt.figure('ex curve',clear='True')
ax = fig.add_subplot(111)
# plt.title('Optimal temperature')
for_detuning = 0
print(deltas[for_detuning]/Gam)

ax.plot(deltas/Gam, T0[:,0],
        marker='+',ls='-',
       label='$\frac{dF}{dT}$')
ax.plot(deltas/Gam, T0[:,1],
        marker='+',ls='-',
       label='$\frac{d^2F}{dT^2}$')

ax.set_xlabel('Detuning [$\Gamma$]')
ax.set_ylabel('Temperature [K]')
ax.grid()
# ax.legend(fontsize=15)
tight_layout()

-1.0


# Second method with temperature variation known

Consider that the temperature variation is known. In our case, with a GiantMolecule with mass $10^6$ Da, charge $Q=ze=e$, injection energy $50$ eV injection velocity $v\approx 100$ m/s. In the cases where detection is delivered, the giant molecule loses 50meV, transfers enough energy to raise the cloud temperature from 1mK to 100mK approximately. For a given $\delta T = 100$ mK, it is possible to compute the $\Delta P$, for different initial temperatures.

It is also possible to compute with the higher $\Delta T = 100$ K as if the simulation worked.

When taking final temperature 0.1 K, the optimum initial temperature that provides the higher fluorescence difference, between initial and final, is 68 mK, which is around the temperature where fluo is maximum. This is not surprising at all because obviously if one wants the highest fluo diff, starts from the highest fluo and go to the lower, which is shown below.

In [239]:
# plot P = MB x DopProfile

vmin = -20 # from 
vmax = 20  # to
nv = 5000  # resolution
Tinit = 0.005   # temperature
Tfinal = 10   # temperature
deltaT = Tfinal-Tinit
delta = -Gam

nunuinit, dopinit = prob_fluo(vmin, vmax, nv, Tinit, delta, k, Rab, Gam)
nunufinal, dopfinal = prob_fluo(vmin, vmax, nv, Tfinal, delta, k, Rab, Gam)

fig = plt.figure('MB x DopProfile two temperatures',clear='True')
ax = fig.add_subplot(111)
ax.plot(nunuinit, dopinit,label=f'{Tinit} K')
ax.plot(nunufinal, dopfinal,label=f'{Tfinal} K')

ax.set_xlabel(r'$\nu = k v$ [Hz]')
ax.set_ylabel(r'Probability')

ax.grid()
ax.legend(loc=3,fontsize=15)

tight_layout()

In [240]:
temps2 = linspace(.001, Tfinal, nv)  # temperatures
fluo_diff = [] # Here is stored the integral MBxDopProfile over detunings
for i, tempinit in enumerate(temps2):
    fluo_diff.append([])    
    for j, delta in enumerate(deltas):
        nunuinit, dopinit = prob_fluo(vmin, vmax, nv, tempinit, delta, k, Rab, Gam)
        nunufinal, dopfinal = prob_fluo(vmin, vmax, nv, Tfinal, delta, k, Rab, Gam)
        fluo_diff[i].append(sum(dopfinal)*dv-sum(dopinit)*dv) # *dv
fluo_diff = array(fluo_diff)

In [170]:
# The ideal initial temperature
# Giving the highest fluo difference
# When going from that initial temp
# to the defined final temp

print(temps2[argmin(fluo_diff[:,51//2])]*1e3,'mK')
print('This value for the initial temp providing highest fluo diff with final temp 10 K')
print('equals the temp for max fluo computed first above')
print('171.548 mK')

171.01700340068015 mK
This value for the initial temp providing highest fluo diff with final temp 10 K
equals the temp for max fluo computed first above
171.548 mK


In [161]:
# plot P = MB x DopProfile

fig = plt.figure('fluo_diff',clear='True')
ax = fig.add_subplot(111)
ax.plot(temps2, fluo_diff[:,51//2])
# ax.vlines(T_lim,-0.0015,0.0015)

ax.set_xlabel(r'$T_{init}$ [K]')
ax.set_ylabel(r'$\Delta F$')

ax.grid()
# ax.legend(loc=3,fontsize=15)

tight_layout()